W pierwszej kolejności pobieram dane z Kaggle odnoszące się do danych marketingowych 
https://www.kaggle.com/datasets/volodymyrgavrysh/bank-marketing-campaigns-dataset

In [ ]:
import pandas as pd
import numpy as np
import os

data_path = r"E:\page_ds\tweaking_the_response_results"



dane = pd.read_csv(os.path.join(data_path, "bank-additional-full.csv"), sep=';')

# przyglądnę się danym i ewentualnie zredukuję listę zmiennych
from dataprep.eda import create_report

report = create_report(dane, title='EDA dla kampanii marketingowej')
# przenoszę do przeglądarki
report.save(os.path.join(data_path, "EDA_dla_kampanii.html"))
report.show_browser()
report.show()



In [6]:
display(dane)

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41183,73,retired,married,professional.course,no,yes,no,cellular,nov,fri,...,1,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,yes
41184,46,blue-collar,married,professional.course,no,no,no,cellular,nov,fri,...,1,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,no
41185,56,retired,married,university.degree,no,yes,no,cellular,nov,fri,...,2,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,no
41186,44,technician,married,professional.course,no,no,no,cellular,nov,fri,...,1,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,yes


Akcje:
1. kodujemy _pdays_ na wartości 1 - gdy =999, 0 w przeciwnym wypadku
2. OHE dla zmiennych tekstowych
3. wywalam zmienne silnie skorelowane - emp.var.rate, euribor3m
4. month - wydaje się popsuta
5. duration - jest sugestia żeby nie dawac do modelu - zbyt silna zależnośc do y (brak kontaktu -> = 0 )

In [17]:
#dane numeryczne
dane._get_numeric_data().columns.difference(['pdays','emp.var.rate','euribor3m'])


Index(['age', 'campaign', 'cons.conf.idx', 'cons.price.idx', 'duration',
       'nr.employed', 'previous'],
      dtype='object')

In [15]:
dane.select_dtypes(include=['object']).columns.difference(['y']).tolist()

['contact',
 'day_of_week',
 'default',
 'education',
 'housing',
 'job',
 'loan',
 'marital',
 'month',
 'poutcome']

In [ ]:
# dodanie zmiennych kategorycznych, robiÃ„Â™ to w pÃ„Â™tli ale moÃ…Â¼na skorzystaÃ„Â‡ rÃƒÂ³wniez z metody OneHotEncoding
cat_vars=['Gender', 'Region_Code', 'Occupation', 'Channel_Code', 'Credit_Product', 'Is_Active']
for var in cat_vars:
    cat_list = pd.get_dummies(credit_cards_data[var], prefix=var)
    data1 = credit_cards_data.join(cat_list)
    credit_cards_data = data1